# Bonobos coding challenge

## Introduction

As a retail company, our goal is to maximize the relevance of each interaction
a customer has with our brand. We can use data to contribute to this effort by
generating personalized content for customers, for example in the form of
outfit recommendations.

The two attached files encode information about products & customer purchases.

* item_attr.tsv       40 item vectors, 4 qualitative attributes
* custo_item.tsv      100 customer vectors, up to 10 purchases

Write a Python script that uses this data to build personalized outfit
recommendations by following the steps below. Keep in mind potential
optimizations to minimize time & space complexity as you design your solution.

Outfits should contain one product from each category (top/bottom/accessory).

* step 1: Map the data in item_attr.tsv into an item-attribute space with binary (0/1)features.
* step 2: Map the data in custo_item.tsv into a customer-item space with binary (0/1) features.
* step 3: Map revealed customer preferences to inferred customer preferences.
* step 4: Create outfit recommendations for 10 customers using their inferred preferences and the following criteria:
    * outfit 1:   biz formal, chilly
    * outfit 2:   date night, warm
    * outfit 3:   casual, hot

Please submit your code and a brief write-up (max 2 pages) of your methodology.
How would your solution scale if the number of items and customers both increased
by several orders of magnitude? What is the most efficient way to store information
about inferred customer preferences? What could you do to minimize the time
between updates to customer preferences? How does the cold start problem
appear in this data, and what could you do to address it? What additional
data do you think might be useful in building this solution, if any?



## Setting the environment and loading data

In [1]:
import pandas as pd
import random
import csv
from sklearn import metrics

In [27]:
# Loading functions
def construct_line(line, item_list):
    # Substitute categoricals with 1/0
	new_line = [line[0]] + ['0']*len(items)	
	for item in line[1:]:
	    index = item_list.index(item)
	    new_line[index] = '1'
	    
	new_line = " ".join( new_line )
	new_line += "\n"
	return new_line

def best_match(x,lst):
    # Best match for recommendation. In case of tie, 
    # take a random draw
    if max(x) == 0:
        ind = random.choice(list(lst))
    else:
        cols = lst[x == max(x)]
        ind = random.choice(cols)
    return ind

def binarize(path):
    # Read the (subset of) customers purchase history from file,
    # returns a dataframe with binary labels
    f = open(path, 'rb')
    custTmp  = csv.reader(f,delimiter='\t')
    custTmp.next() 
    
    # The encoded output will be written here. Header comes from item attribute
    o = open( 'bonobos_ds/tmp.data', 'wb' )
    o.write(header)
    
    # Loop but first set the new customers, for which there is no purchase yet
    new_cust=[]
    while True:
        # read until EOF
        try:
            cust = custTmp.next()  
        except StopIteration:
            break
            
    # if the customer has a purchase history, binarize otherwise save index        
        if len(cust) > 1:
            enc_cust = construct_line(cust, encoded_customer_header)             
            o.write(enc_cust)
        else:
            new_cust.append(cust[0])
            continue        
    f.close()
    o.close()
    
    df_enc = pd.read_csv('bonobos_ds/tmp.data', sep=' ',
                            header=0, index_col=0)
    return [new_cust, df_enc]

def profile(df):
    # calculates user profile dependent on the item features
    
    total_purchase = df.apply(sum,axis=1)
    return df.dot(onehot_items).div(total_purchase,axis = 0)
    
    
def recommender(user_profile, old_cust, new_cust, dress_code=None, temp=None):
    # Calculates the recommendation for the outfit.
    
    # 0- Creates a mask for item already purchased by customers. 
    # These items are filteres out from the final recommendation
    
    mask = -(old_cust-1)
    
    # Filter products according to preferences
    filt_items = onehot_items
    if dress_code != None:
       dress_code = 'dress_code_' + dress_code
       filt_items = onehot_items.loc[onehot_items[dress_code] == 1,:]
       
    if temp != None:
       temp = 'temp_' + temp
       filt_items = filt_items.loc[filt_items[temp] == 1,:]
       
    #1- Recommendation for top
    try:
        filt_items_top = filt_items.loc[filt_items.category_top == 1,:]
        simil_top = metrics.pairwise.cosine_similarity(user_profile, filt_items_top, dense_output=True)
        simil_top = pd.DataFrame(data=simil_top, index=user_profile.index,
                                 columns=filt_items_top.index).mul(mask[filt_items_top.index]) 
        recom_top = simil_top.apply(best_match,axis=1,lst=simil_top.columns)
    except ValueError:
        recom_top = pd.DataFrame(data=[base_recommendation[0]]*len(user_profile.index), index=user_profile.index)
    
    #2- Recommendation for bottom
    try:
        filt_items_bot = filt_items.loc[filt_items.category_bottom == 1,:]
        simil_bot = metrics.pairwise.cosine_similarity(user_profile, filt_items_bot, dense_output=True)
        simil_bot = pd.DataFrame(data=simil_bot, index=user_profile.index, 
                                 columns=filt_items_bot.index).mul(mask[filt_items_bot.index])
        recom_bot = simil_bot.apply(best_match,axis=1,lst=simil_bot.columns)
    except ValueError:
        recom_bot = pd.DataFrame(data=[base_recommendation[1]]*len(user_profile.index), index=user_profile.index)
    
    #3- Recommendation for accessory
    try:
        filt_items_acc = filt_items.loc[filt_items.category_accessory == 1,:]
        simil_acc = metrics.pairwise.cosine_similarity(user_profile, filt_items_acc, dense_output=True)
        simil_acc = pd.DataFrame(data=simil_acc, index=user_profile.index, 
                                 columns=filt_items_acc.index).mul(mask[filt_items_acc.index])
        recom_acc = simil_acc.apply(best_match,axis=1,lst=simil_acc.columns)
    except ValueError:
        recom_acc = pd.DataFrame(data=[base_recommendation[2]] * len(user_profile.index), index=user_profile.index)
    
    recommendations = pd.concat([recom_top, recom_bot,recom_acc], axis = 1)
    recommendations.columns = ['Top', 'Bottom', 'Accessory']
    
    # Now let's provide a recommendation for customer with no purchase history.   
    recommendations_newcust = pd.DataFrame([base_recommendation] * len(new_cust),
                    columns=['Top', 'Bottom', 'Accessory'],index=new_cust )
                    
    return pd.concat([recommendations, recommendations_newcust], axis=0)

def bonobos_recommender(path,dress_code=None,temp=None):
    # Wrapper for the functions described above. Given a tcv 
    # file with customer purchase history and (optionally)
    # a preference on the features, the function provides
    # a recommended outfit. If the request produces no result
    # the base recommendation is used
    
    [new_cust,old_cust] = binarize(path)
    df_usr_profile = profile(old_cust)
    return recommender(df_usr_profile, old_cust, new_cust, dress_code, temp)


## Step 1: One-hot-encoding over the item-attr dataset
In order to scale effectively with the size of the datasets, we need to know: 
1. the number of items available;
2. the number of unique features, 

then we can do label binarization on both datasets. Since we are not given the number of unique attributes, we have to load the entire table item_attr. In a real situation, such attributes are already available (attributes are defined upfront, for product classification) and the one-hot encoding can be done line by line as the new products are available. The updates to the encoded item/attribute table can be appended in real time and time complexity increases with the number of updates. Given the high sparsity of the data, a sparse notation such as libsvm reduces the size of the storage.

In [6]:
# Step 0: Load data for items and attributes
items = pd.read_csv('bonobos_ds/item_attr.tsv', sep='\t', header = 0, index_col=0)
# Step 1: Apply One-hot-encoding
onehot_items = pd.get_dummies(items[['dress_code', 'temp', 'category']])
# Produce the list of unique items, to binarize the customer-item table
items_list = list(items.index)

print 'First 5 entries of items db'
print items.iloc[:5,:7]
onehot_items.head()


First 5 entries of items db
        dress_code     temp   category
item                                  
item_1  date night   chilly        top
item_2  biz casual     warm  accessory
item_3   black tie      hot  accessory
item_4  date night  perfect  accessory
item_5      casual     cold  accessory


,dress_code_biz casual,dress_code_biz formal,dress_code_black tie,dress_code_casual,dress_code_date night,dress_code_wedding,dress_code_weekend,temp_chilly,temp_cold,temp_hot,temp_perfect,temp_warm,category_accessory,category_bottom,category_top
item,,,,,,,,,,,,,,,
item_1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1
item_2,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0
item_3,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0
item_4,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
item_5,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0


## Step 2: One-hot-encoding over the customer-item dataset
As the number of customers is the largest dimension of the problem, customers are read line by line, to improve performance. By using the unique names for items as provided by item-attr, the categorical values of customer-item  are encoded into a binary label. The resulting dataset can be saved (appended) to the encoded customer list. As for the items, writing the output in sparse notation enormously reduces the size of the file.

In [18]:
# define encoded custo-item header
encoded_customer_header = ['customer'] + items_list
header = " ".join(encoded_customer_header)
header += "\n"

# label binarization
path = './bonobos_ds/custo_item.tsv'
[new_cust,old_cust] = binarize(path)

print old_cust.iloc[:10,:7]


          item_1  item_2  item_3  item_4  item_5  item_6  item_7
customer                                                        
cust_1         0       1       0       0       0       0       0
cust_2         0       1       0       0       0       0       0
cust_3         0       0       0       0       0       0       0
cust_4         0       0       0       1       0       1       0
cust_5         0       0       0       0       0       0       0
cust_6         0       0       0       0       0       0       0
cust_7         0       0       0       1       0       0       0
cust_8         0       0       0       0       0       0       0
cust_9         0       0       0       0       0       0       0
cust_10        0       0       0       0       1       0       0


## Step 3: Create user profile and first prediction
For this problem we are given binay preferences for items purchased by a customer and binary attributes related to each item. This suggests using a content based recommender system to infer for the customer preferences. The algorithm only relies on the item features, therefore it does not require to compare a user among peers. This makes it easier to interpret, it allows new/unpopular products to be recommended and it fits better the customer "unique" taste. 
Conversely, the model does not capture the interation between features or groups of users; for example, a new user having an empty profile cannot be initialized without using information from the peers, such as the best_seller item for each category. 
 

In [10]:
# Calculate the base recommendation, defined as the most frequently 
# sold item in each relevant category, top, bottom and accessory. This 
# approximation will be used when the user is new (i.e. no prior purchase)
# or when no item satisfies the search, due to the preferences 
# (dress code, temperature)

best_seller = old_cust.apply(sum,axis=0)
best_seller_top = best_seller.loc[items.category == 'top']
best_seller_top = best_match(best_seller_top, lst=best_seller_top.index)

best_seller_bot = best_seller.loc[items.category == 'bottom']
best_seller_bot = best_match(best_seller_bot, lst=best_seller_bot.index)

best_seller_acc = best_seller.loc[items.category == 'accessory']
best_seller_acc = best_match(best_seller_acc, lst=best_seller_acc.index)

base_recommendation = [best_seller_top, best_seller_bot, best_seller_acc]

print base_recommendation

['item_7', 'item_20', 'item_38']


Given the limited size of the dataset, we can create a user profile by taking the internal product of the customer history and product features, normalized by the number of purchased items. For a general customer update, this is not necessary. As the prediction for user A is independent from user B we can build a function calculating the user profile for the single user.

As the number of products and features increases, so does the time and space complexity. Not all features although are equally relevant. Depending on the relevant (time) window chosen for purchase history or the definition of meta-feautures, the item-features dataset can be partitioned before being used to update the user profile.

In [32]:
# Calculate the user profile as internal product of encoded customers
# and encoded items, normalized by the number of purchases per customer
df_usr_profile = profile(old_cust)

# For each relevant customer profile, calculate the cosine similarity
# with the encoded items and pick the most similar item per relevant 
# category (top, bottom, accessory). Filter out already purchased items
recommendations = recommender(df_usr_profile, old_cust, new_cust)

# Notice that the last 4 rows are new customers, thereofore the base
# recommendation was used to infer their preferences
print '************************'
print 'User profile'
print '************************'
print df_usr_profile.iloc[:10,:3]

print '************************'
print 'Outfit recommendations (no constraint)'
print '************************'
print recommendations.iloc[:6,] 
print recommendations.iloc[-4:,]

************************
User profile
************************
          dress_code_biz casual  dress_code_biz formal  dress_code_black tie
customer                                                                    
cust_1                 0.200000               0.000000              0.000000
cust_2                 0.250000               0.250000              0.250000
cust_3                 0.333333               0.000000              0.333333
cust_4                 0.000000               0.000000              0.333333
cust_5                 0.000000               0.333333              0.000000
cust_6                 0.000000               0.000000              0.000000
cust_7                 0.200000               0.000000              0.200000
cust_8                 0.000000               0.250000              0.250000
cust_9                 0.000000               0.000000              0.500000
cust_10                0.000000               0.000000              0.000000
*************

## Step 4:
Finally, we create an outfit recommendations for 10 customers using their inferred preferences and the following criteria:
    * outfit 1:   biz formal, chilly
    * outfit 2:   date night, warm
    * outfit 3:   casual, hot
Restricting the item list by using dress code and temperature dramatically reduces the number of available items. This has two effects, namely to make the cosine similarity less relevant (many customers share the same recommendation) and  to cause a category (top, bottom or accessory) to be left empty (no item for that combination). In this case the base recommendation is used. 

As the calculation is nearly identical to the previous one, I used the wrapper bonobos_recommender to hide the details.

In [29]:
path = './bonobos_ds/custo_item.tsv'
outfit_1 = bonobos_recommender(path, 'biz formal', 'chilly')
outfit_2 = bonobos_recommender(path, 'date night', 'warm')
outfit_3 = bonobos_recommender(path, 'casual', 'hot')
print '************************'
print 'base recommendation'
print '************************'
print base_recommendation

print '************************'
print 'outfit_1'
print '************************'
print outfit_1.iloc[:10,]

print '************************'
print 'outfit_2'
print '************************'
print outfit_2.iloc[:10,]

print '************************'
print 'outfit_3'
print '************************'
print outfit_3.iloc[:10,]

************************
base_recommendation
************************
['item_7', 'item_20', 'item_38']
************************
outfit_1
************************
             Top   Bottom Accessory
cust_1   item_26  item_17   item_38
cust_2   item_26  item_17   item_38
cust_3   item_26  item_17   item_38
cust_4   item_26  item_17   item_38
cust_5   item_26  item_17   item_38
cust_6   item_26  item_17   item_38
cust_7   item_26  item_17   item_38
cust_8   item_16  item_17   item_38
cust_9   item_26  item_17   item_38
cust_10  item_26  item_17   item_38
************************
outfit_2
************************
             Top   Bottom Accessory
cust_1   item_34  item_35   item_38
cust_2   item_34  item_35   item_38
cust_3   item_34  item_36   item_38
cust_4   item_34  item_35   item_38
cust_5   item_34  item_36   item_38
cust_6   item_34  item_36   item_38
cust_7   item_34  item_36   item_38
cust_8   item_34  item_35   item_38
cust_9   item_34  item_36   item_38
cust_10  item_34  item_